In [145]:
# Read all tracks

import pandas as pd

tracks_df = pd.read_json("tracks.json", lines=True).set_index('track')

In [146]:
# Capture sessions

import pandas as pd
import glob


df = pd.read_json("data.0.json", lines=True)

# df = pd.concat([
#     pd.read_json(data_path, lines=True)
#     for data_path
#     in glob.glob("../rec_sys_data/*/data.json")
# ])

df['ts'] = pd.to_datetime(df['timestamp'], unit='ms')
df       = df.sort_values(['user', 'ts']).reset_index(drop=True)

df['prev_was_last'] = (
    df.groupby('user')['message']
      .shift()
      .eq('last')
      .fillna(False)
)

df['session_id'] = (
    df.groupby('user')['prev_was_last']
      .cumsum()
      .astype(int)
)

In [147]:
df

,message,timestamp,user,track,time,latency,recommendation,experiments,ts,prev_was_last,session_id
0,next,2025-04-16 09:54:42.542,0,48730,1.00,0.000468,5459.0,{'ALL': 'T6'},2025-04-16 09:54:42.542,False,0
1,next,2025-04-16 09:54:42.545,0,5459,0.00,0.000399,5459.0,{'ALL': 'T6'},2025-04-16 09:54:42.545,False,0
2,next,2025-04-16 09:54:42.548,0,5459,0.00,0.000504,40528.0,{'ALL': 'T6'},2025-04-16 09:54:42.548,False,0
3,next,2025-04-16 09:54:42.550,0,40528,0.00,0.000355,41501.0,{'ALL': 'T6'},2025-04-16 09:54:42.550,False,0
4,next,2025-04-16 09:54:42.552,0,41501,0.00,0.000700,22460.0,{'ALL': 'T6'},2025-04-16 09:54:42.552,False,0
...,...,...,...,...,...,...,...,...,...,...,...
1091944,next,2025-04-16 09:58:13.192,9999,10901,0.05,0.000642,15206.0,{'ALL': 'T5'},2025-04-16 09:58:13.192,False,17
1091945,next,2025-04-16 09:58:13.196,9999,15206,0.17,0.000526,47776.0,{'ALL': 'T5'},2025-04-16 09:58:13.196,False,17
1091946,next,2025-04-16 09:58:13.200,9999,47776,0.28,0.000835,1688.0,{'ALL': 'T5'},2025-04-16 09:58:13.200,False,17
1091947,next,2025-04-16 09:58:13.202,9999,1688,0.99,0.000286,6675.0,{'ALL': 'T5'},2025-04-16 09:58:13.202,False,17


In [148]:
# Encode track labels in min time listened descending order for consistency

sessions = df.groupby(['user', 'session_id'])

max_listened = df.groupby('track').max('time')

tracks_ml_df = tracks_df.join(max_listened[['time']].rename(columns={'time': 'max_listened'}), 'track')
tracks_ml_df = tracks_ml_df.sort_values('max_listened', ascending=False)
tracks_ml_df.insert(0, 'track_label', range(len(tracks_ml_df)))

tracks_ml_df

,track_label,artist,album,title,genre,pop,duration,max_listened
track,,,,,,,,
41164,0,Михаил Бублик,ART-Обстрел I-часть,Сорок тысяч верст,"[1, 47]",-0.500252,282,1.0
25058,1,ALEX ANDREEV,Baby Love,Baby Love,[1],-0.844809,180,1.0
3747,2,ROZA FILBERG,Танцевать танго,Танцевать танго,[147],-0.668377,266,1.0
46470,3,Сборная Союза,Бог по имени любовь,"С Днём рождения, девочка моя",[1],0.089019,239,1.0
13981,4,BLVCK PUMV,PHARAON,PHARAON,[36],-0.923226,162,1.0
...,...,...,...,...,...,...,...,...
23194,49995,Азамат Исенгазин,Журавли,Журавли,[147],-0.917465,242,NaN
39122,49996,Nicoleta Nuca,Castele De Nisip,Castele De Nisip,[1],-1.005924,219,NaN
8879,49997,ANNA BELAYA,Неважно,Неважно,[1],-0.652991,154,NaN


In [149]:
sessions = list(\
    df\
    [df['time'] > 0.5]
    # [df['experiments'].apply(lambda x: x['HW2'] == 'T1')]
    .groupby(['user', 'session_id'])\
    ['track']\
    .apply(lambda a: list(map(str, a)))\
    .to_numpy()\
)

sessions = list(filter(lambda s: len(s) > 1, sessions))
len(sessions)

66404

In [150]:
from gensim.models import Word2Vec, KeyedVectors
import pickle as pkl
from gensim.models.callbacks import CallbackAny2Vec
import pathlib

class Saver(CallbackAny2Vec):
    def __init__(self, prefix="song2vec_ckpt", keep_kv_only=True):
        self.prefix = pathlib.Path(prefix)
        self.keep_kv_only = keep_kv_only
        self.epoch = 0
        self.prefix.mkdir(exist_ok=True)          # create folder once

    def on_epoch_end(self, model):
        fname = self.prefix / f"epoch{self.epoch:02d}.model"
        if self.keep_kv_only:
            model.wv.save(str(fname.with_suffix('.kv')))
        else:
            model.save(str(fname))                     # full model inc. optimizer
        print(f"\u2705  saved checkpoint: {fname}")
        self.epoch += 1

import os
if os.path.isfile("w2v.model"):
    w2v = Word2Vec.load('w2v.model')
    w2v.train(sessions, callbacks=[Saver(prefix="song2vec_ckpt_v2", keep_kv_only=False)], total_examples=w2v.corpus_count, epochs=25)
else:
    w2v = Word2Vec(sessions, vector_size=128, sg=1,
                window=10, negative=10, epochs=50,
                min_count=5, sample=1e-3, workers=12,
                callbacks=[Saver(prefix="song2vec_ckpt_v2", keep_kv_only=False)])

w2v.save('w2v.model')
w2v.wv.save('w2v.kv')
print(w2v.wv)



✅  saved checkpoint: song2vec_ckpt_v2/epoch00.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch01.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch02.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch03.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch04.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch05.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch06.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch07.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch08.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch09.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch10.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch11.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch12.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch13.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch14.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch15.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch16.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch17.model
✅  saved checkpoint: song2vec_ckpt_v2/epoch18.model
✅  saved che

In [151]:
from gensim.models import KeyedVectors

kv = KeyedVectors.load('w2v.kv')

def show(p, tid):
    track = tracks_ml_df.loc[tid]
    print(f"{p:>20} {tid:>4} {track['artist']:<20} {track['title']}")

def show_similar(tid, topn=10):
    # print(tracks_ml_df.loc[tid])
    track = tracks_ml_df.loc[int(tid)]
    print(f"Similar to {track['artist']} - {track['title']}:")
    for t in kv.most_similar(positive=[str(tid)], topn=topn):
        track = tracks_ml_df.loc[int(t[0])]
        print(f"[{t[1]}] {t[0]}: {track['artist']} - {track['title']}")

# list(kv.key_to_index.keys())
show_similar(34403)

Similar to Boney M. - Sunny (Live):
[0.949935793876648] 34407: Boney M. - Gotta Go Home (Live)
[0.9327317476272583] 34406: Boney M. - Belfast (Live)
[0.9053914546966553] 34404: Boney M. - Brown Girl in the Ring (Live)
[0.8902466297149658] 34409: Boney M. - Hooray, Hooray, It's a Holi-Holiday (Live)
[0.8897335529327393] 42921: Arabesque - Born To Reggae
[0.8287602066993713] 34408: Boney M. - Ma Baker (Live)
[0.7539907097816467] 36022: Jay Leemo - Я это ты
[0.7454770803451538] 4921: САМУРАЙКИ - САНТО СТЕФАНО
[0.745183527469635] 2004: Клава Кока - Нет проблем
[0.7412952780723572] 46252: Selena Gomez - Slow Down
